In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.데이터 불러오기
1. '연구자 선정'
2. '상관관계 5'
3. '상관관계10'
4. '전체변수'
5. '(연구자선정).동일년도 외부변수, 과거예산포함'
6.  '(연구자선정).동일연도 외부변수, 과거예산 불포함 '
7.  '(상관관계).동일년도 외부변수, 과거예산포함'
8.  '(상관관계).동일연도 외부변수, 과거예산 불포함'
9. '33년 ARIMA'

In [ ]:
#1. 파일 생성
writer=pd.ExcelWriter('lasso변수선택결과.xlsx', engine='openpyxl')

In [ ]:
# file_name_list =['과학기술 아리마', '과학기술 샤프_장기추계_ori', '고등교육 아리마', '고등교육 샤프_장기추계_ori','기후대기 아리마', '기후대기 샤프_장기추계_ori','식품 아리마', '식품 샤프_장기추계_ori']
file_name_list =['찐_부문 고등교육(최종)', '찐_부문 기후대기(최종)', '찐_분야 과학기술(최종)', '찐_프로그램 식품(최종)']

file_name = file_name_list[1]
print(file_name)
excel_sheets = pd.read_excel(f'/content/drive/MyDrive/PC - Desktop/재정전망_동국대/최종데이터분석/3.데이터/{file_name}.xlsx', sheet_name = None)


찐_부문 기후대기(최종)


In [ ]:
sheets = list(excel_sheets.keys())
print(sheets)
file = excel_sheets[sheets[3]]
# file.drop('Unnamed: 0', axis = 1, inplace=True)
file.head()

['연구자 선정', '상관관계5', '상관관계10', '모든 변수']


,Unnamed: 0,기후대기및환경안전,정치변수,생산자물가지수(전체),소비자물가지수(전체),고령인구비율(65세 이상),국내총생산(실질),기대여명,추계인구,의료보건(가계목절별 최종소비지출_실질),...,연료전지(수소/신에너지),신재생에너지 생산량,재생에너지 생산량,이륜차신고현황,호흡기질환 진료,호흡기질환 진료비 합계,에너지소비량_합계,에너지소비량_석탄,에너지소비량_석유,에너지소비량_천연가스
0,2009,317945,0,88.09,78.010,9.8,12864585,79.2,48683638,223357,...,1832,5608776,5606944,1785051,27416969,2527924248,236486,59653,105718,34471
1,2010,285123,0,95.63,81.656,10.2,13252193,79.6,49054708,240724,...,4367,5858479,5854112,1814399,28621396,4324246641,240994,66100,100373,35671
2,2011,301026,1,95.42,83.906,10.5,13357243,80.0,49307835,260735,...,19193,6086254,6067061,1820729,30737810,5055429153,243500,68595,102535,33908
3,2012,323322,1,99.06,86.373,10.8,14266180,80.2,49554112,277285,...,42347,6856419,6814072,1825474,30207251,5117745080,264053,77142,104500,43008
4,2013,291913,1,105.71,89.850,11.0,14791984,80.6,49936638,292973,...,63345,7584042,7520697,1828312,29526664,5173212782,276998,83687,105459,46284


## 2. 분석 시작


*   데이터셋을 window2로 만들어주는 def
*   모델들



In [ ]:
final_model = []
real_value = []
pred_value = []

In [ ]:
# 정규화 적용
def scaler(data):
    first_two_columns = data.iloc[:, :2]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.iloc[:, 2:])
    scaled_data_df = pd.DataFrame(scaled_data, columns=data.columns[2:])
    result = pd.concat([first_two_columns, scaled_data_df], axis=1)
    result.columns = data.columns
    # print(result)
    return result

scaler(file)

,Unnamed: 0,기후대기및환경안전,정치변수,생산자물가지수(전체),소비자물가지수(전체),고령인구비율(65세 이상),국내총생산(실질),기대여명,추계인구,의료보건(가계목절별 최종소비지출_실질),...,연료전지(수소/신에너지),신재생에너지 생산량,재생에너지 생산량,이륜차신고현황,호흡기질환 진료,호흡기질환 진료비 합계,에너지소비량_합계,에너지소비량_석탄,에너지소비량_석유,에너지소비량_천연가스
0,2009,317945,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.631291,0.000000,0.000000,0.000000,0.274793,0.021683
1,2010,285123,0.0,0.350535,0.148877,0.058824,0.061306,0.090909,0.117703,0.063840,...,0.002482,0.020056,0.021105,0.058235,0.722455,0.358161,0.063236,0.238301,0.000000,0.067899
2,2011,301026,1.0,0.340772,0.240751,0.102941,0.077921,0.181818,0.197994,0.137399,...,0.016999,0.038350,0.039288,0.070796,0.882648,0.503949,0.098390,0.330524,0.111151,0.000000
3,2012,323322,1.0,0.509995,0.341486,0.147059,0.221683,0.227273,0.276113,0.198236,...,0.039670,0.100208,0.103074,0.080211,0.842489,0.516374,0.386699,0.646448,0.212174,0.350472
4,2013,291913,1.0,0.819154,0.483463,0.176471,0.304847,0.318182,0.397450,0.255904,...,0.060230,0.158649,0.163412,0.085842,0.790975,0.527433,0.568286,0.888371,0.261478,0.476642
5,2014,266496,1.0,0.853092,0.563699,0.250000,0.361056,0.386364,0.480941,0.321578,...,0.078997,0.260828,0.270405,0.611986,0.867348,0.560845,0.586971,0.775597,0.296180,0.626882
6,2015,339025,0.0,0.774059,0.612495,0.308824,0.436875,0.500000,0.553592,0.392361,...,0.118071,0.344145,0.355575,0.658753,0.839239,0.512143,0.605305,0.809159,0.264357,0.716927
7,2016,496454,0.0,0.748024,0.660923,0.382353,0.516027,0.590909,0.654387,0.463204,...,0.193419,0.478719,0.492073,0.696554,0.937347,0.669554,0.644807,0.914689,0.219577,0.533988
8,2017,653826,0.0,0.553696,0.688077,0.441176,0.587681,0.659091,0.739487,0.525831,...,0.223586,0.621805,0.641452,0.747529,0.884159,0.696206,0.708296,0.951726,0.448152,0.373387
9,2018,861250,0.0,0.469084,0.725725,0.500000,0.664960,0.727273,0.803833,0.612689,...,0.234736,0.695550,0.712491,0.785059,0.964957,0.827297,0.804946,0.807496,0.885918,0.447140


In [ ]:
def make_window_size2(data):

    raw = []
    days_in = 2

    for i in range(days_in, 0, -1):
        raw.append(data.shift(i-1))

    prep_inde = pd.concat(raw, axis=1)
    prep_inde.dropna(inplace=True)
    prep_inde.reset_index(drop=True, inplace=True)
    print(prep_inde)
    return prep_inde

In [ ]:
def pred_2123(dataset):
  X_train = dataset.iloc [:11,:]
  y_train = file.iloc[1:12,1].values
  X_test = dataset.iloc [11:,:]
  y_test = file.iloc[12:,1].values
  print(X_train)
  print(y_train)
  print(X_test)
  print(y_test)
  return X_train, y_train, X_test, y_test

def pred_2021a(dataset):
  X_train = dataset.iloc [:11,:]
  y_train = file.iloc[1:12,1].values
  X_test = dataset.iloc [11:,:]
  y_test = file.iloc[12:,1].values
  print(X_train)
  print(y_train)
  print(X_test)
  print(y_test)
  return X_train, y_train, X_test, y_test

def pred_2021b(dataset):
  X_train = dataset.iloc [:12,:]
  y_train = file.iloc[1:13,1].values
  X_test = dataset.iloc [12:,:]
  y_test = file.iloc[13:,1].values
  print(X_train)
  print(y_train)
  print(X_test)
  print(y_test)
  return X_train, y_train, X_test, y_test

In [ ]:
 # window_size = 2 데이터세트 준비
scaler_data = scaler(file)
win2_set = make_window_size2(scaler_data.iloc[:,:].astype(float))
# win2_set.to_csv('win2.csv', encoding='cp949')
X_train, y_train, X_test, y_test = pred_2123(win2_set)

    Unnamed: 0  기후대기및환경안전  정치변수  생산자물가지수(전체)  소비자물가지수(전체)  고령인구비율(65세 이상)  \
0       2009.0   317945.0   0.0     0.000000     0.000000        0.000000   
1       2010.0   285123.0   0.0     0.350535     0.148877        0.058824   
2       2011.0   301026.0   1.0     0.340772     0.240751        0.102941   
3       2012.0   323322.0   1.0     0.509995     0.341486        0.147059   
4       2013.0   291913.0   1.0     0.819154     0.483463        0.176471   
5       2014.0   266496.0   1.0     0.853092     0.563699        0.250000   
6       2015.0   339025.0   0.0     0.774059     0.612495        0.308824   
7       2016.0   496454.0   0.0     0.748024     0.660923        0.382353   
8       2017.0   653826.0   0.0     0.553696     0.688077        0.441176   
9       2018.0   861250.0   0.0     0.469084     0.725725        0.500000   
10      2019.0  2358215.0   0.0     0.626685     0.801756        0.588235   
11      2020.0  2804333.0   1.0     0.715481     0.860596        0.661765   

In [ ]:
# 1.Lasso 모델
lasso = Lasso()
param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 3, 5, 10, 100]}
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.073e+07, tolerance: 3.433e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.362e+09, tolerance: 6.969e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.852e+09, tolerance: 7.350e

최적의 하이퍼파라미터: {'alpha': 100}
[Lasso(alpha=100)]
[array([3415247, 4146772, 4529212])]
[array([3413647.19725925, 4260713.77403921, 4410921.30546093])]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.277e+07, tolerance: 3.433e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.269e+09, tolerance: 8.056e+08
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# 2. Ridge 모델
ridge = Ridge()
param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 3, 5, 10, 100]}
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=2.88091e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=1.12692e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=5.16022e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=3

최적의 하이퍼파라미터: {'alpha': 0.0001}
[Lasso(alpha=100), Ridge(alpha=0.0001)]
[array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212])]
[array([3413647.19725925, 4260713.77403921, 4410921.30546093]), array([3415246.99999983, 4146771.99999977, 4529211.99999978])]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=2.04869e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)


In [ ]:
# 3. Linear 모델
linear = LinearRegression()
linear.fit(X_train, y_train)
predictions = linear.predict(X_test)

final_model.append(linear)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

[Lasso(alpha=100), Ridge(alpha=0.0001), LinearRegression()]
[array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212])]
[array([3413647.19725925, 4260713.77403921, 4410921.30546093]), array([3415246.99999983, 4146771.99999977, 4529211.99999978]), array([3415246.99999983, 4146771.99999977, 4529211.99999978])]


In [ ]:
# 4. RandomForest 모델
random_forest = RandomForestRegressor(random_state=42)

# 탐색할 하이퍼파라미터 그리드
param_grid = [
    {},  # 아무런 하이퍼파라미터가 없는 경우
    {'n_estimators': [10, 50, 100]},
    {'max_depth': [None, 10, 20]},
    {'min_samples_split': [2, 5, 10]},
    {'min_samples_leaf': [1, 2, 4]}
]

grid_search = GridSearchCV(random_forest, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

최적의 하이퍼파라미터: {'n_estimators': 50}
[Lasso(alpha=100), Ridge(alpha=0.0001), LinearRegression(), RandomForestRegressor(n_estimators=50, random_state=42)]
[array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212])]
[array([3413647.19725925, 4260713.77403921, 4410921.30546093]), array([3415246.99999983, 4146771.99999977, 4529211.99999978]), array([3415246.99999983, 4146771.99999977, 4529211.99999978]), array([2072004.6 , 1857222.94, 1733504.  ])]


In [ ]:
# 5. GB 모델
gradient_boosting = GradientBoostingRegressor(random_state=42)

# 탐색할 하이퍼파라미터 그리드
param_grid = [
    {},  # 아무런 하이퍼파라미터가 없는 경우
    {'n_estimators': [10, 50, 100]},
    {'learning_rate': [0.01, 0.1, 0.2]},
    {'max_depth': [3, 5, 7]},
    {'min_samples_split': [2, 5, 10]},
    {'min_samples_leaf': [1, 2, 4]}
]

# GridSearchCV를 사용하여 최적의 하이퍼파라미터를 탐색
grid_search = GridSearchCV(gradient_boosting, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

final_model.append(best_model)
real_value.append(y_test)
pred_value.append(predictions)
print(final_model)
print(real_value)
print(pred_value)

최적의 하이퍼파라미터: {}
[Lasso(alpha=100), Ridge(alpha=0.0001), LinearRegression(), RandomForestRegressor(n_estimators=50, random_state=42), GradientBoostingRegressor(random_state=42)]
[array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212])]
[array([3413647.19725925, 4260713.77403921, 4410921.30546093]), array([3415246.99999983, 4146771.99999977, 4529211.99999978]), array([3415246.99999983, 4146771.99999977, 4529211.99999978]), array([2072004.6 , 1857222.94, 1733504.  ]), array([2102452.1538993 , 1777541.64209374, 1223630.3646474 ])]


In [ ]:
print(file_name)
print(final_model)
print(real_value)
print(pred_value)

찐_부문 기후대기(최종)
[Lasso(alpha=100), Ridge(alpha=0.0001), LinearRegression(), RandomForestRegressor(n_estimators=50, random_state=42), GradientBoostingRegressor(random_state=42)]
[array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212]), array([3415247, 4146772, 4529212])]
[array([3413647.19725925, 4260713.77403921, 4410921.30546093]), array([3415246.99999983, 4146771.99999977, 4529211.99999978]), array([3415246.99999983, 4146771.99999977, 4529211.99999978]), array([2072004.6 , 1857222.94, 1733504.  ]), array([2102452.1538993 , 1777541.64209374, 1223630.3646474 ])]


In [ ]:
def df_2123(df_results, file_name, final_model, real_value, pred_value):
    start_index = 0

    for model_index in range(len(final_model)):
        for j in range(len(real_value[model_index])):
            df_results.loc[start_index] = [file_name, final_model[model_index], real_value[model_index][j], pred_value[model_index][j]]
            start_index += 1

    return df_results


def df_2021(df_results, file_name, final_model, real_value, pred_value):
    start_index = 0

    for i, model_index in enumerate(range(5)):
        for j in range(len(real_value[model_index])):
            df_results.loc[start_index] = [file_name, final_model[model_index], real_value[model_index][j], pred_value[model_index][j]]
            start_index += 1

    return df_results


def df_23n(df_results, file_name, final_model, real_value, pred_value):
    start_index = 0

    for i, model_index in enumerate(range(5)):
        for j in range(len(real_value[model_index])):
            df_results.loc[start_index] = [file_name, final_model[model_index], real_value[model_index][j], pred_value[model_index][j]]
            start_index += 1

    return df_results


# 테이블 열 이름 정의
columns = ['file_name', '최종모델', '실제값', '예측값']

# 2021년, 2022년, 2023년 데이터를 위한 빈 데이터프레임 생성
df_results = pd.DataFrame(columns=columns)

final_dataset = df_2123(df_results, file_name, final_model, real_value, pred_value)
df_results.to_excel(writer, sheet_name=f'lasso변수선택_{file_name}')

In [ ]:
#3. 작성 완료 후 파일 저장
writer.close()